In [1]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
import time

In [3]:
上次count = 0
def 记录(milliseconds, count):
    h,m,s,ms = 时间点(milliseconds)
    msg = "{}:{:0>2d}:{:0>2d}.{:0>3d},{}\n".format(h,m,s,ms,count)
    file1.write(msg)
def 记录2(milliseconds, count):
    # global 上次count
    if abs(count - 上次count) > 1024:
        h,m,s,ms = 时间点(milliseconds)
        msg = "{}:{:0>2d}:{:0>2d}.{:0>3d}\n".format(h,m,s,ms)
        file2.write(msg)
    # 上次count = count
def 时间点(milliseconds):
    ms = milliseconds % 1000
    ms = ms // 1
    s  = milliseconds // 1000
    m  = s // 60
    s  = s % 60
    h  = m // 60
    m  = m % 60
    
    return int(h),int(m),int(s),int(ms)
def 做差(f1, c1, c2 = np.array([255,255,255]), threshold = 128):
    # f21 = np.concatenate( (f1[:,2:,:], f1[:,:-2,:]), axis = 2 )
    # f22 = np.concatenate( (f1[2:,:,:], f1[:-2,:,:]), axis = 2 )
    f2 = np.concatenate( (f1[:,2:,:], f1[:,:-2,:]), axis = 2 )
    
    f31 = f2 - np.concatenate( (c1,c2) )
    f41 = np.sum(np.abs(f31),axis=2)
    f51 = f41 < threshold
    
    f32 = f2 - np.concatenate( (c2,c1) )
    f42 = np.sum(np.abs(f32),axis=2)
    f52 = f42 < threshold

    f61 = f51+f52
    x, y = f61.shape
    f71 = f61.repeat(3).reshape(x, y, 3)

    f2 = np.concatenate( (f1[2:,:,:], f1[:-2,:,:]), axis = 2 )
    
    f31 = f2 - np.concatenate( (c1,c2) )
    f41 = np.sum(np.abs(f31),axis=2)
    f51 = f41 < threshold
    
    f32 = f2 - np.concatenate( (c2,c1) )
    f42 = np.sum(np.abs(f32),axis=2)
    f52 = f42 < threshold
    
    f62 = f51+f52
    x, y = f62.shape
    f72 = f62.repeat(3).reshape(x, y, 3)

    count = f71.sum()+f72.sum()
    return f72, f71, count

In [4]:
file1 = open( 'file.txt','w+', encoding = 'utf-8')
file2 = open( 'logfile.txt','w+', encoding = 'utf-8')

In [5]:
cameraCapture = cv2.VideoCapture('./079.mp4')

In [6]:
success = True
preFrame = np.zeros((120,1280,3), dtype=np.uint8)

lastEdge = 0
上次count = 0 
上升list = []
下降list = []
def 记录count(count):
    if   ( (count-上次count) > 192 ):
        lastEdge = 1
        h,m,s,ms = 时间点(milliseconds)
        msg = "{}:{:0>2d}:{:0>2d}.{},{:0>3d}\n".format(h,m,s,ms,count)
        上升list.append(msg)
    elif ( (count-上次count) < -192 ):
        lastEdge = -1
        h,m,s,ms = 时间点(milliseconds)
        msg = "{}:{:0>2d}:{:0>2d}.{},{:0>3d}\n".format(h,m,s,ms,count)
        下降list.append(msg)

In [7]:
# 抓轴过程
while success:
    if cv2.waitKey(1) == 27:
        break
    success, frame = cameraCapture.read()
    milliseconds = cameraCapture.get(cv2.CAP_PROP_POS_MSEC)

    canvas = np.zeros((720,1280,3), dtype=np.uint8) # 过程查看

    # 字幕区
    subFrame = frame[600:720,:]
    canvas[0:120,:,:] = subFrame[:,:,:] # 过程查看

    # 求差
    x,y, count = 做差(subFrame.copy(), np.array([189,44,20]))

    记录(milliseconds, count)
    记录2(milliseconds, count)
    # cv2.imshow('Test camera', canvas)
    上次count = count 
cv2.destroyAllWindows()
    

TypeError: 'NoneType' object is not subscriptable

In [ ]:
file2.close()

In [9]:
# 收尾
file1.close()
file2.close()

下面是测试内容

In [ ]:
while success:
    # if cv2.waitKey(1) == 27:
    #     break
    success, frame = cameraCapture.read()
    milliseconds = cameraCapture.get(cv2.CAP_PROP_POS_MSEC)

    canvas = np.zeros((1080-120,1280,3), dtype=np.uint8)
    # canvas[240:360,:,:] = preFrame[:,:,:]
    # preFrame[:,:,:] = frame[600:720,:]

    subFrame = frame[600:720,:]
    canvas[0:120,:,:] = subFrame[:,:,:]

    lbimg=cv2.medianBlur(subFrame,3)
    canvas[120:240,:,:] = lbimg[:,:,:]

    sub1 = cv2.subtract(subFrame,preFrame)
    sub2 = cv2.subtract(preFrame,subFrame)
    canvas[360:480,:,:] = sub1[:,:,:]
    canvas[480:600,:,:] = sub2[:,:,:]

    shiro = 魔棒(subFrame.copy(), np.array([255,255,255]) )
    canvas[600:720,:,:] = shiro[:,:,:]
    pink = 魔棒(subFrame.copy(), np.array([171,0,231]) ) # BGR  × RGB
    canvas[720:840,:,:] = pink[:,:,:]

    x, count = 差(subFrame.copy(), np.array([171,0,231]), np.array([255,255,255]))
    canvas[840:960,1:-1,:] = x[:,:,:]*255

    preFrame = subFrame.copy()
    cv2.imshow('Test camera', canvas)
    # print(int(milliseconds))
    file.write("{0},{1}\n".format(int(milliseconds),count.sum()))

cv2.destroyAllWindows()

In [ ]:
# im = subFrame
while success:
    if cv2.waitKey(1) == 27:
        break
    cv2.imshow('', im)
cv2.destroyAllWindows()

In [ ]:
def 魔棒(aI, color, threshold = 128):
    t1 = aI - color
    t2 = np.sum(np.abs(t1),axis=2)
    t3 = t2 < threshold
    t4 = t3 * 255
    x, y = t4.shape
    t5 = t4.repeat(3).reshape(x, y, 3)
    return t5
def 差(aI, c1, c2, threshold = 128):
    f1 = aI
    f2 = np.concatenate( (f1[:,2:,:], f1[:,:-2,:]), axis = 2 )
    
    f31 = f2 - np.concatenate( (c1,c2) )
    f41 = np.sum(np.abs(f31),axis=2)
    f51 = f41 < threshold
    
    f32 = f2 - np.concatenate( (c2,c1) )
    f42 = np.sum(np.abs(f32),axis=2)
    f52 = f42 < threshold

    f5 = f51+f52

        



    x, y = f5.shape
    f6 = f5.repeat(3).reshape(x, y, 3)
    return (f6, f5)

In [ ]:
f.close()

In [ ]:
B = np.arange(1,5).reshape(2,2)  #原始输入数组

In [ ]:
cv2.destroyAllWindows()

In [ ]:
im = f6*255

In [ ]:
# canvas[840:960,1:-1,:] = f2[:,:,:3]
c2 = np.array([171,0,231])
c1 = np.array([255,255,255])

In [ ]:
a.shape

In [ ]:

    # seconds = milliseconds//1000
    # milliseconds = milliseconds%1000
    # minutes = 0
    # hours = 0
    # if seconds >= 60:
    #     minutes = seconds//60
    #     seconds = seconds % 60

    # if minutes >= 60:
    #     hours = minutes//60
    #     minutes = minutes % 60
    # print(frame)